### Importing necessary libraries & dataset

In [1]:
import tensorflow as tf
import pandas as pd

In [2]:
tf.__version__

'2.8.0'

In [3]:
loan = pd.read_csv(r'C:\Users\SHREE\Downloads\Datasets\personal loan_updated.csv')
loan.head()

,ID,Age,Experience,Income,ZIP Code,Family,CCAvg,Education,Mortgage,Personal Loan,Securities Account,CD Account,Online,CreditCard
0,1,25,1,49,91107,4,1.6,1,0,0,1,0,0,0
1,2,45,19,34,90089,3,1.5,1,0,0,1,0,0,0
2,3,39,15,11,94720,1,1.0,1,0,0,0,0,0,0
3,4,35,9,100,94112,1,2.7,2,0,0,0,0,0,0
4,5,35,8,45,91330,4,1.0,2,0,0,0,0,0,1


### Data PreProcessing

In [4]:
loan.isnull().sum()

ID                    0
Age                   0
Experience            0
Income                0
ZIP Code              0
Family                0
CCAvg                 0
Education             0
Mortgage              0
Personal Loan         0
Securities Account    0
CD Account            0
Online                0
CreditCard            0
dtype: int64

In [5]:
loan.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 14 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   ID                  5000 non-null   int64  
 1   Age                 5000 non-null   int64  
 2   Experience          5000 non-null   int64  
 3   Income              5000 non-null   int64  
 4   ZIP Code            5000 non-null   int64  
 5   Family              5000 non-null   int64  
 6   CCAvg               5000 non-null   float64
 7   Education           5000 non-null   int64  
 8   Mortgage            5000 non-null   int64  
 9   Personal Loan       5000 non-null   int64  
 10  Securities Account  5000 non-null   int64  
 11  CD Account          5000 non-null   int64  
 12  Online              5000 non-null   int64  
 13  CreditCard          5000 non-null   int64  
dtypes: float64(1), int64(13)
memory usage: 547.0 KB


In [8]:
# We will remove ID & Zip code for model building as it will be unique
loan['ZIP Code'].nunique()

467

In [9]:
loan_final = loan.drop(columns = ['ID', 'ZIP Code'])

In [10]:
loan.shape

(5000, 14)

In [11]:
loan_final.shape

(5000, 12)

### Identifying target variables, inputs & splitting them into train, test

In [12]:
y = loan_final[['Personal Loan']]
x = loan_final.drop(columns = ['Personal Loan'])

In [13]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 42)
x_train.shape, x_test.shape, y_train.shape, y_test.shape

((4000, 11), (1000, 11), (4000, 1), (1000, 1))

### Step 1 - Initialising the model & creating input layer

In [15]:
ann = tf.keras.models.Sequential()

### Step 2 - Creating the first hidden layer

In [17]:
ann.add(tf.keras.layers.Dense(units = 6, activation = 'relu'))

### Step 3 - Creating the second hidden layer

In [18]:
ann.add(tf.keras.layers.Dense(units = 6, activation = 'relu'))

### Step 4 - Creating the output layer

In [19]:
# As our target variable has 2 classes, we have to fix units = 1 (n-1)
ann.add(tf.keras.layers.Dense(units = 1, activation = 'sigmoid'))

### Step 5 - Compiling the model

In [20]:
ann.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = 'accuracy') # if we have multiclass classification then we can use categorical cross entropy

### Step 6 - Fitting the data

In [21]:
model = ann.fit(x_train, y_train, validation_split = 0.2, batch_size = 32, epochs = 100)

Epoch 1/100
100/100 [==============================] - 5s 8ms/step - loss: 0.7089 - accuracy: 0.9025 - val_loss: 0.3202 - val_accuracy: 0.8838
Epoch 2/100
100/100 [==============================] - 0s 4ms/step - loss: 0.2741 - accuracy: 0.8916 - val_loss: 0.2967 - val_accuracy: 0.8900
Epoch 3/100
100/100 [==============================] - 0s 3ms/step - loss: 0.2576 - accuracy: 0.8956 - val_loss: 0.2918 - val_accuracy: 0.8975
Epoch 4/100
100/100 [==============================] - 0s 3ms/step - loss: 0.2502 - accuracy: 0.8984 - val_loss: 0.2848 - val_accuracy: 0.9000
Epoch 5/100
100/100 [==============================] - 0s 3ms/step - loss: 0.2462 - accuracy: 0.9003 - val_loss: 0.2786 - val_accuracy: 0.8963
Epoch 6/100
100/100 [==============================] - 0s 3ms/step - loss: 0.2412 - accuracy: 0.9003 - val_loss: 0.2773 - val_accuracy: 0.9000
Epoch 7/100
100/100 [==============================] - 0s 3ms/step - loss: 0.2373 - accuracy: 0.9022 - val_loss: 0.2723 - val_accuracy: 0.8988

### Step 7 - Prediction on test data

In [23]:
y_test['Prediction'] = ann.predict(x_test)

In [24]:
y_test['Prediction'] = y_test['Prediction'] > 0.5
y_test.head()

,Personal Loan,Prediction
1501,0,False
2586,1,False
2653,0,False
1055,0,False
705,0,False


In [25]:
y_test['Prediction'] = y_test['Prediction'].astype('int')
y_test.head()

,Personal Loan,Prediction
1501,0,0
2586,1,0
2653,0,0
1055,0,0
705,0,0


### Checking Accuracy of model

In [26]:
from sklearn.metrics import accuracy_score, confusion_matrix

print(accuracy_score(y_test['Personal Loan'], y_test['Prediction']))
print(confusion_matrix(y_test['Personal Loan'], y_test['Prediction']))

0.949
[[884  11]
 [ 40  65]]
